https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/select-fields?utm_source=chatgpt.com

# Cấu hình Postgres

In [15]:
DB_CONFIG = {
    "dbname":"openalexdb",
    "user":"postgres",
    "password":"123",
    "host":"localhost",
    "port":"5432"
}   

# Lấy dữ liệu về Works

In [28]:
from pyalex import Works,Authors,Institutions,config
import psycopg2
from psycopg2.extras import execute_values
config.email = "thuattruongminh@gmail.com"

1) Tạo bảng

In [17]:
def create_table():
    connect = psycopg2.connect(**DB_CONFIG)
    cursor = connect.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS works(
            id TEXT PRIMARY KEY,
            title TEXT,
            authorships TEXT,
            concepts TEXT,
            publication_date TEXT,
            doi TEXT
        )
        """)
    connect.commit()
    cursor.close()
    connect.close()

2. Lưu vào postgres

In [39]:
def save_to_postgres(records):
    connect = psycopg2.connect(**DB_CONFIG)
    cursor = connect.cursor()
    rows = []
    for r in records:
        authorships = ", ".join([a["author"]["display_name"] for a in r.get("authorships", [])]) or None
        concepts = ", ".join([c["display_name"] for c in r.get("concepts", [])]) or None
        rows.append((
            r.get("id"),
            r.get("title"),
            authorships,
            concepts,
            r.get("publication_date"),
            r.get("doi")
        ))
    execute_values(cursor,"""
                   INSERT INTO works(id,title,authorships,concepts,publication_date,doi)
                   VALUES %s
                   ON CONFLICT (id) DO NOTHING
                   """,rows)
    connect.commit()
    cursor.close()
    connect.close()
    

3. Call API

In [37]:
def crawl_and_save(pages = 2):
    fields = "id,title,authorships,concepts,publication_date,doi"
    for page in range(1,pages+1):
        print(f"Crawling page {page}...")
        data = Works().filter(type = "article").select(fields).get(per_page = 200,page = page)
        save_to_postgres(data)
    print("Done!")

In [40]:
if __name__ == "__main__":
    create_table()
    crawl_and_save(pages = 2)

Crawling page 1...
Crawling page 2...
Done!
